In [133]:
import pandas as pd
from pymongo import MongoClient
import requests
from dotenv import load_dotenv
load_dotenv()



True

In [134]:
client = MongoClient("mongodb://localhost:27017/")

In [135]:
db = client.companies 

In [136]:
companies = db.companies.find()

In [137]:
data_companies = pd.DataFrame(companies)

In [138]:
data_companies.head(3)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,video_embeds,screenshots,external_links,partners,founded_month,founded_day,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,...,"[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],9.0,15.0,NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],10.0,17.0,NaN,NaN,NaN,NaN


In [150]:
#esto irá con mis propios criterios y con inputs

companies = db.companies.find({"$and":[
                                            {"offices": {"$exists": True}},
                                            {"offices": {"$ne": None}}, 
                                            {"$and" : [{"offices.latitude": {"$exists": True}}, {"offices.longitude": {"$exists": True}}]},
                                            {"category_code": {"$exists": True}},
                                            {"category_code": {"$ne": None}}, 
                                            {"founded_year": {"$exists": True}}, 
                                            {"founded_year": {"$gte": 2003}},
                                            {"deadpooled_year": None},
                                            {"number_of_employees": {"$exists": True}},
                                            {"number_of_employees": {"$gte": 10}},
                                            {"total_money_raised": {"$exists": True}},
                                            {"total_money_raised":{"$ne":None}},
                                            {"total_money_raised": {"$not":{"$size":0}}}, 
                                            
                                            ]
                                            },

                                            # descartamos elementos que no interesan
                                            {"_id": 0, "crunchbase_url": 0, "products": 0, 
                                             "acquisition": 0, "acquisitions": 0, "video_embeds": 0, 
                                             "screenshots": 0, "external_links": 0, "partners": 0, 
                                             "image": 0, 
                                             'deadpooled_day': 0, 'deadpooled_month': 0, 
                                             'deadpooled_url': 0, 'deadpooled_year': 0,
                                            }
                                     )


In [151]:
companies = pd.DataFrame(companies)
offices = companies.set_index('name').offices.apply(pd.Series).stack().reset_index(level=-1, drop=True).reset_index()
offices.columns = ['name', 'office']
companies = companies.drop('offices', 1)
companies = companies.merge(offices, on='name', how='outer')

#intentar aplicarlo directamente con apply
#dfx['B'].apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame('B').join(dfx[['A']], how='left')
#funcion?

In [152]:
companies.head(3)

,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,...,overview,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,milestones,ipo,office
0,Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,6.0,...,<p>Geni is an online community of casual and e...,"[{'is_past': False, 'title': 'CEO', 'person': ...","[{'competitor': {'name': 'Ancestry', 'permalin...",[],$16.5M,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",[],"[{'id': 15460, 'description': 'Announced hire ...",None,"{'description': 'Headquarters', 'address1': '9..."
1,Digg,digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60,2004,10.0,...,<p>Digg is a user driven social content websit...,"[{'is_past': False, 'title': 'CEO', 'person': ...","[{'competitor': {'name': 'Reddit', 'permalink'...","[{'title': 'Public Relations', 'is_past': True...",$45M,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...",[],"[{'id': 9588, 'description': 'Another Digg Exe...",None,"{'description': None, 'address1': '135 Mississ..."
2,Twitter,twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300,2006,3.0,...,"<p>Created in 2006, Twitter is a global real-t...","[{'is_past': False, 'title': 'CEO', 'person': ...","[{'competitor': {'name': 'Digg', 'permalink': ...","[{'title': 'Hosting', 'is_past': True, 'provid...",$1.16B,"[{'id': 261, 'round_code': 'a', 'source_url': ...",[],"[{'id': 1, 'description': 'Survives iPhone 3G ...","{'valuation_amount': 18100000000, 'valuation_c...","{'description': '', 'address1': '1355 Market S..."


In [153]:
def createLat(office):
    return office['latitude']
def createLon(office):
    return office['longitude']

companies['lat'] = companies['office'].apply(createLat)
companies['long'] = companies['office'].apply(createLon)
companies.dropna(subset=['lat', 'long'], inplace=True)

In [154]:
companies.head(3)

,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,...,competitions,providerships,total_money_raised,funding_rounds,investments,milestones,ipo,office,lat,long
0,Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,6.0,...,"[{'competitor': {'name': 'Ancestry', 'permalin...",[],$16.5M,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",[],"[{'id': 15460, 'description': 'Announced hire ...",None,"{'description': 'Headquarters', 'address1': '9...",34.090368,-118.393064
1,Digg,digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60,2004,10.0,...,"[{'competitor': {'name': 'Reddit', 'permalink'...","[{'title': 'Public Relations', 'is_past': True...",$45M,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...",[],"[{'id': 9588, 'description': 'Another Digg Exe...",None,"{'description': None, 'address1': '135 Mississ...",37.764726,-122.394523
2,Twitter,twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300,2006,3.0,...,"[{'competitor': {'name': 'Digg', 'permalink': ...","[{'title': 'Hosting', 'is_past': True, 'provid...",$1.16B,"[{'id': 261, 'round_code': 'a', 'source_url': ...",[],"[{'id': 1, 'description': 'Survives iPhone 3G ...","{'valuation_amount': 18100000000, 'valuation_c...","{'description': '', 'address1': '1355 Market S...",37.776805,-122.416924


In [155]:
def createGeoJson(office):
    return {
        "type":"Point",
        "coordinates":[office["long"],office["lat"]]
    }

companies.dropna(subset=['lat', 'long'], inplace=True)
companies['position'] = companies.apply(createGeoJson, axis=1)
display(companies).head(3)
companies.shape

,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,...,providerships,total_money_raised,funding_rounds,investments,milestones,ipo,office,lat,long,position
0,Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,6.0,...,[],$16.5M,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",[],"[{'id': 15460, 'description': 'Announced hire ...",None,"{'description': 'Headquarters', 'address1': '9...",34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,Digg,digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60,2004,10.0,...,"[{'title': 'Public Relations', 'is_past': True...",$45M,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...",[],"[{'id': 9588, 'description': 'Another Digg Exe...",None,"{'description': None, 'address1': '135 Mississ...",37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
2,Twitter,twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300,2006,3.0,...,"[{'title': 'Hosting', 'is_past': True, 'provid...",$1.16B,"[{'id': 261, 'round_code': 'a', 'source_url': ...",[],"[{'id': 1, 'description': 'Survives iPhone 3G ...","{'valuation_amount': 18100000000, 'valuation_c...","{'description': '', 'address1': '1355 Market S...",37.776805,-122.416924,"{'type': 'Point', 'coordinates': [-122.4169244..."
3,Scribd,scribd,http://scribd.com,http://blog.scribd.com/,http://blog.scribd.com/feed/,scribd,news,50,2007,NaN,...,[],$25.8M,"[{'id': 10, 'round_code': 'seed', 'source_url'...",[],"[{'id': 4960, 'description': 'Scribd launches ...",None,"{'description': 'HQ', 'address1': '539 Bryant ...",37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,..."
4,MeetMoi,meetmoi,http://www.meetmoi.com,,,meetmoi,social,15,2007,NaN,...,[],$5.58M,"[{'id': 18, 'round_code': 'a', 'source_url': '...",[],[],None,"{'description': None, 'address1': '', 'address...",40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ..."
5,Facebook,facebook,http://facebook.com,http://blog.facebook.com,http://blog.facebook.com/atom.php,facebook,social,5299,2004,2.0,...,"[{'title': '', 'is_past': False, 'provider': {...",$2.43B,"[{'id': 2, 'round_code': 'angel', 'source_url'...","[{'funding_round': {'round_code': 'seed', 'sou...","[{'id': 108, 'description': 'Facebook adds com...","{'valuation_amount': 104000000000, 'valuation_...","{'description': 'Headquarters', 'address1': '1...",37.416050,-122.151801,"{'type': 'Point', 'coordinates': [-122.151801,..."
6,Facebook,facebook,http://facebook.com,http://blog.facebook.com,http://blog.facebook.com/atom.php,facebook,social,5299,2004,2.0,...,"[{'title': '', 'is_past': False, 'provider': {...",$2.43B,"[{'id': 2, 'round_code': 'angel', 'source_url'...","[{'funding_round': {'round_code': 'seed', 'sou...","[{'id': 108, 'description': 'Facebook adds com...","{'valuation_amount': 104000000000, 'valuation_...","{'description': 'Europe HQ', 'address1': '', '...",53.344104,-6.267494,"{'type': 'Point', 'coordinates': [-6.267494, 5..."
7,Facebook,facebook,http://facebook.com,http://blog.facebook.com,http://blog.facebook.com/atom.php,facebook,social,5299,2004,2.0,...,"[{'title': '', 'is_past': False, 'provider': {...",$2.43B,"[{'id': 2, 'round_code': 'angel', 'source_url'...","[{'funding_round': {'round_code': 'seed', 'sou...","[{'id': 108, 'description': 'Facebook adds com...","{'valuation_amount': 104000000000, 'valuation_...","{'description': 'New York', 'address1': '340 M...",40.755716,-73.979247,"{'type': 'Point', 'coordinates': [-73.9792469,..."
8,Powerset,powerset,http://powerset.com,http://blog.powerset.com/,http://blog.powerset.com/atom.xml,Powerset,search,60,2006,10.0,...,"[{'title': '', 'is_past': False, 'provider': {...",$22.5M,"[{'id': 22, 'round_code': 'a', 'source_url': '...",[],[],None,"{'description': None, 'address1': '475 Brannan...",37.778

AttributeError: 'NoneType' object has no attribute 'head'

In [156]:
# AQUÍ DEBERÍA LIMPIAR LA BASE DE DATOS: CURRENCY Y M,K..!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# ARREGLAR LAS CATEGORÍAS DE LA BASE DE DATOS PARA AGRUPAR?

In [158]:
#Importar archivo json a Mongo
# mongoimport --db companies --collection selected --jsonArray offices.json


companies_selected.to_json('./offices.json', orient="records")

In [ ]:
#-----------------AQUÍ ACABA LA PARTE DE CLEANSE Y EMPIEZA LA PARTE DE FILTRADO (1 - GEONEAR)-----------------

In [163]:
#Volverlo a traer a Jupyter
client = MongoClient('mongodb://localhost:27017/')
db = client.companies
selected = db.selected.find({"$and" : [{"total_money_raised": {"$exists": True}},
                                                {"$or": [
                                                {"category_code": "design" } ,
                                                {"category_code": "web" } , 
                                                {"category_code": "software" } , 
                                                {"category_code": "games_video" } , 
                                                {"category_code": "mobile" } , 
                                                {"category_code": "enterprise" } ,   
                                                {"category_code": "analytics" } ,
                                                {'category_code': "search"},
                                                {'category_code': "network_hosting"} ,   
                                                {"category_code": "photo_video" } ,   
                                                ]}]})

#AÑADIR QUERY AQUÍ!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! (CON INPUTS!)
#Y CON TOTAL MONEY RAISED YA CON NUMEROS!!!!!!!!!!
#Y SOLAMENTE CON UN CATEGORY CODE (PERO DE LOS AGRUPADOS)!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
selected_df = pd.DataFrame(selected)
#selected_df.dropna(subset=['lat', 'long'], inplace=True)

#display(selected_df.position)
selected_df.head(3)

,_id,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,...,providerships,total_money_raised,funding_rounds,investments,milestones,ipo,office,lat,long,position
0,5d7d39c81b1e66e6d6070bc8,Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,...,[],$16.5M,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",[],"[{'id': 15460, 'description': 'Announced hire ...",None,"{'description': 'Headquarters', 'address1': '9...",34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,5d7d39c81b1e66e6d6070bc9,Mahalo,mahalo,http://mahalo.com,http://blog.mahalo.com/,http://blog.mahalo.com/feed/,MahaloDotCom,web,40,2007,...,[],$21M,"[{'id': 323, 'round_code': 'a', 'source_url': ...",[],[],None,"{'description': '', 'address1': '3525 Eastham ...",34.017606,-118.487267,"{'type': 'Point', 'coordinates': [-118.487267,..."
2,5d7d39c81b1e66e6d6070bca,Powerset,powerset,http://powerset.com,http://blog.powerset.com/,http://blog.powerset.com/atom.xml,Powerset,search,60,2006,...,"[{'title': '', 'is_past': False, 'provider': {...",$22.5M,"[{'id': 22, 'round_code': 'a', 'source_url': '...",[],[],None,"{'description': None, 'address1': '475 Brannan...",37.778613,-122.395289,"{'type': 'Point', 'coordinates': [-122.395289,..."


In [164]:
selected_df.category_code.value_counts()

software           860
web                788
enterprise         340
mobile             280
games_video        248
network_hosting    152
search             132
analytics           68
photo_video         20
design               6
Name: category_code, dtype: int64

In [174]:
#db.selected.createIndex( { "position" : "2dsphere" } )
#como hacer esto desde python en vez de desde 

TypeError: 'Collection' object is not callable. If you meant to call the 'ensureIndex' method on a 'Collection' object it is failing because no such method exists.

In [165]:
#funcion para sacar compañías a la redonda
#crear el geoindex en MongoDB Compass
def getCompaniesNear(lat,long, max_meters=2000):
    return list(db.selected.find({
        "position": {
            "$near": {
               "$geometry": {
                  "type": "Point" ,
                  "coordinates": [ long , lat ]
               },
               "$maxDistance": max_meters
             }
       }
    }))


In [166]:
coord_list = []

for i in selected_df.index:
    coord = selected_df.iloc[i]["position"]['coordinates']
    coord_list.append([str(coord),len(pd.DataFrame(getCompaniesNear(coord[1],coord[0])).index)])

coord_list.sort(key = lambda x: x[1], reverse = True)

In [167]:
coord_list

[['[-122.4041764, 37.7819286]', 178],
 ['[-122.4013217, 37.7814765]', 178],
 ['[-122.4041764, 37.7819286]', 178],
 ['[-122.4013217, 37.7814765]', 178],
 ['[-122.402195, 37.786183]', 176],
 ['[-122.403256, 37.78236]', 176],
 ['[-122.405081, 37.785792]', 176],
 ['[-122.400591, 37.782163]', 176],
 ['[-122.402195, 37.786183]', 176],
 ['[-122.403256, 37.78236]', 176],
 ['[-122.405081, 37.785792]', 176],
 ['[-122.400591, 37.782163]', 176],
 ['[-122.409173, 37.788482]', 174],
 ['[-122.40971, 37.788796]', 174],
 ['[-122.409173, 37.788482]', 174],
 ['[-122.40971, 37.788796]', 174],
 ['[-122.406912, 37.781002]', 172],
 ['[-122.401803, 37.778991]', 172],
 ['[-122.406912, 37.781002]', 172],
 ['[-122.401803, 37.778991]', 172],
 ['[-122.4084994, 37.7909427]', 170],
 ['[-122.4020593, 37.7797255]', 170],
 ['[-122.4084994, 37.7909427]', 170],
 ['[-122.4020593, 37.7797255]', 170],
 ['[-122.402759, 37.787646]', 160],
 ['[-122.402759, 37.787646]', 160],
 ['[-122.404113, 37.789471]', 158],
 ['[-122.404113,

In [92]:
# para luego pasarlo a Tableau!!!!! (incluir el centro en la lista)

"""
m = df_filtered[["category_code","lat","long","name"]]
m = m.append({
    "category_code":"center",
    "lat":fifth_avenue[0],
    "long":fifth_avenue[1],
    "name":"CENTER"
},ignore_index=True)
display(m.head(3))

m.to_json('filtered_data.json', orient="records")

"""

,category_code,lat,long,name
0,software,40.775309,-73.983656,Producteev
1,software,40.775309,-73.983656,Producteev
2,software,40.775309,-73.983656,Producteev


In [ ]:
# poner tb cómo se carga el csv de crunchbase a mongo xt
